In [3]:
%cd F:\project4


F:\project4


In [16]:
from application import db
from application.users.models import User
from cassandra.cqlengine.management import sync_table


In [17]:
db.get_session()
sync_table(User)

In [18]:
#User.create_user("honeysaxena525@gmail.com", password='abc123')

In [12]:
user_a = User.create_user("abc@gmail.com", password='abc123')


Exception: user already has account with this email

In [19]:
user_a.password

'$argon2id$v=19$m=65536,t=3,p=4$pgu5iS/0Ya2a8zxeCcNzBA$CncfEST4KOo0kv6i4kelt5lqM3mdCxjsqiotswJJdwI'

In [21]:
user_a.verify_password('abc123')

True